<a href="https://colab.research.google.com/github/LeonardoSanBenitez/Unlearn-Saliency/blob/master/PEM_composition_img_gen/01_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import sys
import os

is_colab: bool = 'google.colab' in sys.modules

if is_colab:
    # You need to manually create the .env file
    %cd /content
    !pip install python-dotenv
    import dotenv
    dotenv.load_dotenv()

    import os
    assert (os.getenv("HF_TOKEN") is not None) and (len(os.getenv("HF_TOKEN"))>0)

    # TODO: install the dependencies from our repo, not from difusers
    if not os.path.exists('diffusers'):
        !git clone https://github.com/huggingface/diffusers
    %cd diffusers
    !pip install .
    !pip install torchmetrics

    %cd examples/text_to_image
    !pip install -r requirements.txt

    !huggingface-cli login --token ${HF_TOKEN}
    !accelerate config default

    %cd /content/
    if not os.path.exists('Unlearn-Saliency'):
        !git clone https://github.com/LeonardoSanBenitez/Unlearn-Saliency.git
    %cd /content/Unlearn-Saliency/PEM_composition_img_gen
elif "alumnos" in os.getcwd():
    %cd ~
    if not os.path.exists('Unlearn-Saliency'):
        !git clone https://github.com/LeonardoSanBenitez/Unlearn-Saliency.git
    %cd ~/Unlearn-Saliency/PEM_composition_img_gen

    !python3.10 -m pip install poetry~=1.2
    !python3.10 -m poetry config virtualenvs.create true

    if os.path.exists("poetry.lock"):
        !rm poetry.lock
    !sed -i 's/python = "~3.11"/python = "~3.10"/' ./pyproject.toml
    !python3.10 -m poetry install --no-root --no-interaction --no-ansi
    
    import dotenv
    dotenv.load_dotenv()
    assert (os.getenv("HF_TOKEN") is not None) and (len(os.getenv("HF_TOKEN"))>0)

    !huggingface-cli login --token ${HF_TOKEN}
    !accelerate config default
else:
    sys.path.append('..')

In [7]:
model_base_name = "stable-diffusion-v1-5/stable-diffusion-v1-5"

In [8]:
import datasets
from PIL import Image
import matplotlib.pyplot as plt
import random
from diffusers import AutoPipelineForText2Image, StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchmetrics.functional.multimodal import clip_score
from functools import partial
from typing import Literal, List, Dict, Tuple, Optional, Callable, Union

import os
import sys
import shutil

from libs.datasets import UnlearnDatasetImagenette, UnlearnDatasetCifar, UnlearnDatasetSplitMode, UnlearnDatasetSplit


# Download dataset

In [25]:
#if os.path.exists('assets/imagenette_splits'):
#    shutil.rmtree('assets/imagenette_splits')
if not os.path.exists('assets/imagenette_splits'):
    for c in UnlearnDatasetImagenette.class_mapping:
        print('-'*80)
        print('Forgetting', UnlearnDatasetImagenette.class_mapping[c])
        dataset = UnlearnDatasetImagenette(
            split_mode=UnlearnDatasetSplitMode.Class,
            split_kwargs={"forget": [c]},
            download_path="assets/imagenette",
        )
        
        dataset.save(f"assets/imagenette_splits/{c}", format='jpg')

--------------------------------------------------------------------------------
Forgetting tench
Dataset already downloaded and extracted. Skipping download.


# Train

In [ ]:
for c in UnlearnDatasetImagenette.class_mapping:
    print('-'*80)
    print('Training', UnlearnDatasetImagenette.class_mapping[c])
    dataset_forget_name = f"./assets/imagenette_splits/{c}/train_forget"
    dataset_retain_name = f"./assets/imagenette_splits/{c}/train_retain"
    output_dir = f"./assets/lora/imagenette_splits/{c}"
    
    !accelerate launch --mixed_precision="fp16" --num_processes 1 --num_machines 1 --dynamo_backend "no" train_text_to_image_lora_munba.py \
        --pretrained_model_name_or_path={model_base_name} \
        --dataset_forget_name={dataset_forget_name} \
        --dataset_retain_name={dataset_retain_name} \
        --dataloader_num_workers=2 \
        --resolution=512 \
        --mixed_precision "fp16" \
        --train_batch_size=4 \
        --gradient_accumulation_steps=8 \
        --learning_rate=2e-04 \
        --max_grad_norm=1 \
        --lr_scheduler="cosine" \
        --lr_warmup_steps=0 \
        --output_dir={output_dir} \
        --num_train_epochs=5 \
        --validation_epochs=1 \
        --checkpointing_steps=500 \
        --validation_prompt=f"Picture of a {c}" \
        --num_validation_images=1 \
        --seed=42
    #break

--------------------------------------------------------------------------------
Training tench


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/benle1/.local/lib/python3.11/site-packages/phasepack/tools.py:11: UserWarning: 
Module 'pyfftw' (FFTW Python bindings) could not be imported. To install it, try
running 'pip install pyfftw' from the terminal. Falling back on the slower
'fftpack' module for 2D Fourier transforms.
  warnings.warn("""
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
03/21/2025 13:47:39 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'variance_type', 'prediction_type', 'thresholding', 'sample_max_value', 'clip_sample_range', 'dynamic_thresholding_ratio', 'rescale_betas_zero_snr', 'timestep_spacing'} was not found in config. Values will be initialized to default values.
{'use_post_quant_conv', 'mid_block_add_attention', 'scaling_factor', 'latents_s

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/benle1/.local/lib/python3.11/site-packages/phasepack/tools.py:11: UserWarning: 
Module 'pyfftw' (FFTW Python bindings) could not be imported. To install it, try
running 'pip install pyfftw' from the terminal. Falling back on the slower
'fftpack' module for 2D Fourier transforms.
  warnings.warn("""
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
03/21/2025 14:26:20 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'sample_max_value', 'variance_type', 'timestep_spacing', 'clip_sample_range', 'dynamic_thresholding_ratio', 'prediction_type', 'rescale_betas_zero_snr', 'thresholding'} was not found in config. Values will be initialized to default values.
{'scaling_factor', 'use_quant_conv', 'force_upcast', 'latents_mean', 'mid_block

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/benle1/.local/lib/python3.11/site-packages/phasepack/tools.py:11: UserWarning: 
Module 'pyfftw' (FFTW Python bindings) could not be imported. To install it, try
running 'pip install pyfftw' from the terminal. Falling back on the slower
'fftpack' module for 2D Fourier transforms.
  warnings.warn("""
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
03/21/2025 14:49:18 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'prediction_type', 'thresholding', 'variance_type', 'sample_max_value', 'clip_sample_range', 'dynamic_thresholding_ratio', 'rescale_betas_zero_snr', 'timestep_spacing'} was not found in config. Values will be initialized to default values.
{'scaling_factor', 'latents_std', 'shift_factor', 'use_post_quant_conv', 'use_q

# Evaluation

In [ ]:
from libs.metrics.image_and_text import MetricImageTextSimilarity
metric = MetricImageTextSimilarity(metrics=['clip'])

for class_model in UnlearnDatasetImagenette.class_mapping:
    print('-'*80)
    print('Evaluating', UnlearnDatasetImagenette.class_mapping[class_model])

    output_dir = f"./assets/lora/imagenette_splits/{class_model}"
    pipeline_unlearned = AutoPipelineForText2Image.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, safety_checker=None).to("cuda")
    pipeline_unlearned.load_lora_weights(output_dir, weight_name="pytorch_lora_weights.safetensors")

    fig, axes = plt.subplots(1, 10, figsize=(15, 3))
    for col, class_eval in enumerate(UnlearnDatasetImagenette.class_mapping):
        prompt = f"An image of {UnlearnDatasetImagenette.class_mapping[class_eval]}"
        image = pipeline_unlearned(prompt).images[0]
        score_clip: float = metric.score(image, prompt)['clip']

        axes[col].imshow(image)
        axes[col].set_title(f"{UnlearnDatasetImagenette.class_mapping[class_eval]}\nClip={score_clip:.1f}", fontsize=8)
        axes[col].axis("off")
    plt.plot()
    #break